In [ ]:
# ===============================================================
# 🚗 License Plate Detection using YOLOv3 (Darknet-format dataset)
# Dataset: /content/License Plate.v1-yolocharacter.darknet.zip
# ===============================================================

# --- Step 1: Unzip Dataset ---
!unzip "/content/License Plate.v1-yolocharacter.darknet.zip" -d dataset_license

# --- Step 2: Install Dependencies ---
try:
    import ultralytics
except ImportError:
    %pip install -q ultralytics

import os
import time
import glob
import json
import math
import torch
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
import yaml

# --- Step 3: Check Device ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device, torch.cuda.get_device_name(0) if device == "cuda" else "")

# --- Step 4: Dataset YAML Configuration ---
class_names = ['License_Plate']   # single class (whole plate)
# If your dataset labels are per-character, replace with alphanumeric list:
# class_names = ['0','1','2','3','4','5','6','7','8','9',
#                'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

dataset_path = "/content/dataset_license"
data_yaml_path = os.path.join(dataset_path, "data.yaml")

data = {
    'train': f'{dataset_path}/train',
    'val': f'{dataset_path}/valid',
    'test': f'{dataset_path}/test',
    'nc': len(class_names),
    'names': class_names
}

with open(data_yaml_path, 'w') as f:
    yaml.dump(data, f)

# --- Step 5: Configuration ---
DATA_YAML = data_yaml_path
TEST_SOURCE = f"{dataset_path}/test"
EPOCHS = 10
BATCH = 16
SIZES = [416, 608]
RESULTS_CSV = "/content/license_plate_results.csv"
ALT_RESULTS_CSV = "/mnt/data/license_plate_results.csv"
os.makedirs("/content", exist_ok=True)

from pathlib import Path

# --- Step 6: Helper Functions ---
def count_params(model_obj) -> int:
    try:
        return sum(p.numel() for p in model_obj.parameters())
    except Exception:
        return -1

def find_best_weights(save_dir: str) -> str:
    if not save_dir:
        return ""
    p = Path(save_dir) / "weights" / "best.pt"
    return str(p) if p.exists() else ""

def file_size_mb(path: str) -> float:
    try:
        return os.path.getsize(path) / (1024 * 1024.0)
    except Exception:
        return float("nan")

def extract_metrics_dict(val_results):
    out = {"map50": None, "map5095": None, "precision": None, "recall": None, "f1": None}
    if isinstance(val_results, dict):
        out["map50"] = val_results.get("metrics/mAP50(B)", val_results.get("map50"))
        out["map5095"] = val_results.get("metrics/mAP50-95(B)", val_results.get("map"))
        out["precision"] = val_results.get("metrics/precision(B)", val_results.get("precision"))
        out["recall"] = val_results.get("metrics/recall(B)", val_results.get("recall"))
        if out["precision"] and out["recall"]:
            p, r = float(out["precision"]), float(out["recall"])
            out["f1"] = 2 * p * r / (p + r)
        return out

    try:
        m50 = getattr(getattr(val_results, "box", None), "map50", None)
        m5095 = getattr(getattr(val_results, "box", None), "map", None)
        p = getattr(val_results, "precision", None)
        r = getattr(val_results, "recall", None)
        f1 = getattr(val_results, "f1", None)
        out["map50"] = float(m50) if m50 is not None else None
        out["map5095"] = float(m5095) if m5095 is not None else None
        out["precision"] = float(p) if p is not None else None
        out["recall"] = float(r) if r is not None else None
        out["f1"] = float(f1) if f1 is not None else None
        if out["f1"] is None and out["precision"] and out["recall"]:
            P, R = out["precision"], out["recall"]
            out["f1"] = 2 * P * R / (P + R)
    except Exception:
        pass
    return out

def time_training(model, imgsz: int, data_yaml: str, epochs: int, batch: int):
    start = time.time()
    train_results = model.train(data=data_yaml, epochs=epochs, imgsz=imgsz, batch=batch, verbose=True)
    end = time.time()
    train_time_min = (end - start) / 60.0
    save_dir = ""
    try:
        save_dir = str(model.trainer.save_dir)
    except Exception:
        pass
    return train_results, train_time_min, save_dir

def validate_metrics(model, data_yaml: str):
    val_results = model.val(data=data_yaml, verbose=True)
    return extract_metrics_dict(val_results)

def warmup_predict(model, test_source: str):
    try:
        one = None
        if os.path.isdir(test_source):
            exts = ("*.jpg","*.jpeg","*.png","*.bmp")
            files = []
            for e in exts:
                files += glob.glob(os.path.join(test_source, "**", e), recursive=True)
            if files:
                one = files[0]
        if one:
            _ = model.predict(source=one, verbose=False)
    except Exception as e:
        print("Warmup skipped:", e)

def time_inference_folder(model, test_source: str):
    exts = ("*.jpg","*.jpeg","*.png","*.bmp")
    files = []
    if os.path.isdir(test_source):
        for e in exts:
            files += glob.glob(os.path.join(test_source, "**", e), recursive=True)
    else:
        files = [test_source] if os.path.isfile(test_source) else []
    n = len(files)
    if n == 0:
        print(f"No images found in {test_source}. Inference timing will be NaN.")
        return float("nan")
    warmup_predict(model, test_source)
    t1 = time.time()
    _ = model.predict(source=test_source, verbose=False)
    t2 = time.time()
    avg_ms = ((t2 - t1) / n) * 1000.0
    return avg_ms

# --- Step 7: Main Experiment Function ---
def run_experiment(imgsz: int):
    print(f"\n===== Running experiment at imgsz={imgsz} =====")
    model = YOLO("yolov3u.pt")
    model.to(device)

    num_params = count_params(model.model)
    train_results, train_time_min, save_dir = time_training(model, imgsz, DATA_YAML, EPOCHS, BATCH)
    best_w = find_best_weights(save_dir)
    best_size_mb = file_size_mb(best_w) if best_w else float("nan")
    metrics = validate_metrics(model, DATA_YAML)
    avg_ms = time_inference_folder(model, TEST_SOURCE)

    row = {
        "input_size": imgsz,
        "train_time_min": train_time_min,
        "inference_ms_per_image": avg_ms,
        "map50": metrics.get("map50"),
        "map50_95": metrics.get("map5095"),
        "precision": metrics.get("precision"),
        "recall": metrics.get("recall"),
        "f1": metrics.get("f1"),
        "num_params": num_params,
        "best_weights_path": best_w,
        "best_weights_size_mb": best_size_mb,
        "runs_dir": save_dir
    }
    return row

# --- Step 8: Run Experiments for Sizes ---
all_rows = []
for s in SIZES:
    row = run_experiment(s)
    all_rows.append(row)

df = pd.DataFrame(all_rows)
display(df)

# --- Step 9: Save Results ---
df.to_csv(RESULTS_CSV, index=False)
try:
    os.makedirs("/mnt/data", exist_ok=True)
    df.to_csv(ALT_RESULTS_CSV, index=False)
except Exception as e:
    print("Could not save ALT_RESULTS_CSV:", e)

print("Saved results:", RESULTS_CSV)

# --- Step 10: Visualization ---
plt.figure()
plt.plot(df["input_size"], df["map50"], marker="o")
plt.title("mAP50 vs Input Size")
plt.xlabel("Input Size")
plt.ylabel("mAP50")
plt.grid(True)
plt.show()

plt.figure()
plt.plot(df["input_size"], df["train_time_min"], marker="o")
plt.title("Training Time (min) vs Input Size")
plt.xlabel("Input Size")
plt.ylabel("Training Time (min)")
plt.grid(True)
plt.show()

plt.figure()
plt.plot(df["input_size"], df["inference_ms_per_image"], marker="o")
plt.title("Inference Speed (ms/image) vs Input Size")
plt.xlabel("Input Size")
plt.ylabel("ms per image")
plt.grid(True)
plt.show()

# --- Step 11: Test Visualization ---
model_416 = YOLO("/content/runs/detect/train/weights/best.pt")
model_608 = YOLO("/content/runs/detect/train2/weights/best.pt")

import random, glob
test_imgs = random.sample(glob.glob(f"{dataset_path}/test/*.jpg"), 5)

from IPython.display import Image, display
for img in test_imgs:
    r1 = model_416.predict(img, save=False)
    r2 = model_608.predict(img, save=False)
    display(r1[0].plot())
    display(r2[0].plot())
